## **SJF, SRTF, FCFS, Round Robin (RR), Priority Scheduling (PS) Algorithms**

In [3]:
from collections import deque

sample_processes=[ #Sample dataset with pid, arrival time, burst time, and priority (can add more PIDs or the values can be changed)
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results): #Function to print scheduling results in a table
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}") #Print the table with fixed column widths
    for r in results: #Loop through each result entry (each process)
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def sjf(sample_processes):
    time = 0 #Current simulation time
    results = [] #List to store scheduling results
    sample_processes = sample_processes.copy() #Copy to avoid modifying original list
    done = [] # Tracks completed processes

    while len(done) < len(sample_processes): #Continue until all processes are finished
        available = [p for p in sample_processes if p["arrival"] <= time and p not in done]

        if not available: # If no process is ready at this time, advance time until something arrives
            time += 1
            continue

        p = min(available, key=lambda x: x["burst"]) #Choose process with smallest burst time (SJF rule)

        start = time #Process starts when CPU is free
        finish = time + p["burst"] #Finish time based on burst length
        waiting = start - p["arrival"] #Time spent waiting in ready queue
        turnaround = finish - p["arrival"] #Total time from arrival to completion

        results.append({"pid": p["pid"], "start": start, "finish": finish, "waiting": waiting, "turnaround": turnaround})

        time = finish #Move simulation time to process completion
        done.append(p) #Mark process as completed

    return results

def srtf(sample_processes):
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" field to track remaining burst
    time = 0 #Current simulation time
    complete = 0 #Number of completed processes
    n = len(sample_processes) #Total number of processes
    results = {} #Stores results sorted by PID
    current = None #Currently running process

    while complete < n: #Continue until all processes finish
        available = [p for p in sample_processes if p["arrival"] <= time and p["remaining"] > 0] #All arrived processes with remaining CPU time

        if not available: # If nothing has arrived yet or all are finished, advance time until something becomes available
            time += 1
            continue

        p = min(available, key=lambda x: x["remaining"]) #Choose process with the "shortest remaining time"

        if p["pid"] not in results: #First time this process gets CPU
            results[p["pid"]] = {"pid": p["pid"], "start": time} #Its first start time (could be preempted later)

        p["remaining"] -= 1 #Execute process for 1 unit of time (preemptive)
        time += 1 #Move the time time forward

        if p["remaining"] == 0: #If process has finished
            results[p["pid"]]["finish"] = time #Record finish time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting time = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround time = finish - arrival
            complete += 1

    return list(results.values()) #Convert dict to list

def fcfs(sample_processes):
    time = 0 #Current simulation time
    results = [] #Stores scheduling results

    sample_processes = sorted(sample_processes, key=lambda x: x["arrival"]) #Sort processes by arrival time (FCFS rule)

    for p in sample_processes: #Process each job in order of arrival
        start = max(time, p["arrival"]) #CPU starts when free OR when process arrives
        finish = start + p["burst"] #Completion time based on burst duration
        waiting = start - p["arrival"] #Waiting = time before first getting CPU
        turnaround = finish - p["arrival"] #Turnaround = total time from arrival to finish

        results.append({"pid": p["pid"], "start": start, "finish": finish, "waiting": waiting, "turnaround": turnaround}) #Add process results to output list
        time = finish #Update time to when this process finishes
    return results

def round_robin(sample_processes, quantum): #RR scheduling with time quantum
    queue = deque() #Queue to hold ready processes (First In First Out)
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" burst time for preemption
    time = 0 #Current simulation time
    results = {} #Stores results by PID

    sample_processes = sorted(sample_processes, key=lambda x: x["arrival"]) #Sort by arrival time for processing
    
    i = 0
    while i < len(sample_processes) or queue: #Continue while processes remain or queue not empty
        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time:
            queue.append(sample_processes[i]) #Add arriving process to ready queue
            i += 1

        if not queue: #If queue empty, jump time to next arrival
            time = sample_processes[i]["arrival"]
            continue

        p = queue.popleft() #Get next process in RR order

        if p["pid"] not in results: #If first time running the process, record its first start time
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        run_time = min(quantum, p["remaining"]) #Execute up to quantum or remaining time
        p["remaining"] -= run_time #Reduce remaining burst
        time += run_time #Advance simulation time

        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time: #Add any newly arrived processes during this time slice
            queue.append(sample_processes[i])
            i += 1

        if p["remaining"] > 0: # If process still not finished, requeue it to get another time slice
            queue.append(p)
        else: # If finished, record completion time
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround = finish - arrival

    return list(results.values()) #Convert dict to list

def priority_non_preemptive(sample_processes):
    time = 0 #Current simulation time
    results = [] #Stores scheduling results
    done = [] #Tracks completed processes

    while len(done) < len(sample_processes): #Loop until all processes are finished
        available = [p for p in sample_processes if p["arrival"] <= time and p not in done] #Get all processes that have arrived and are not completed

        if not available: # If nothing is ready to run, advance time until a process arrives
            time += 1
            continue

        p = min(available, key=lambda x: x["priority"]) #Select process with highest priority (lowest priority number)

        start = time #Process starts as soon as CPU is free
        finish = start + p["burst"] #Completion time = start + burst time
        waiting = start - p["arrival"] #Waiting time = start - arrival
        turnaround = finish - p["arrival"] #Turnaround = finish - arrival

        results.append({"pid": p["pid"], "start": start, "finish": finish, "waiting": waiting, "turnaround": turnaround}) #Appending scheduling outcome

        time = finish
        done.append(p) #Mark process as completed

    return results

def priority_preemptive(sample_processes):
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" burst to allow preemption
    time = 0 #Current simulation time
    n = len(sample_processes) #Total number of processes
    complete = 0 #Count of finished processes
    results = {} #Store results by PID

    while complete < n: #Run until all processes complete
        available = [p for p in sample_processes if p["arrival"] <= time and p["remaining"] > 0] #Get all arrived processes with remaining CPU time

        if not available: #If nothing is ready yet, skip time until a process appears
            time += 1
            continue

        p = min(available, key=lambda x: x["priority"]) #Pick the process with the highest priority (lowest priority number)

        if p["pid"] not in results: #If this is the first time running the process, record first time it touches the CPU
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        p["remaining"] -= 1 #Run for 1 time unit
        time += 1 #Move time forward

        if p["remaining"] == 0: #If process finishes now, record finish time
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround = finish - arrival
            complete += 1 #Increment completed count

    return list(results.values()) #Convert dict to list

print("SJF:")
print_schedule(sjf(sample_processes))

print("\nSRTF:")
print_schedule(srtf(sample_processes))

print("\nFCFS:")
print_schedule(fcfs(sample_processes))

print("\nRound Robin (q=3):")
print_schedule(round_robin(sample_processes, quantum=3))

print("\nPriority (Non-Preemptive):")
print_schedule(priority_non_preemptive(sample_processes))

print("\nPriority (Preemptive):")
print_schedule(priority_preemptive(sample_processes))

SJF:
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P4    16     23     13      20        
P3    23     34     21      32        

SRTF:
PID   Start  Finish Waiting Turnaround
P1    0      23     13      23        
P2    1      7      0       6         
P4    7      14     4       11        
P3    23     34     21      32        

FCFS:
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P3    16     27     14      25        
P4    27     34     24      31        

Round Robin (q=3):
PID   Start  Finish Waiting Turnaround
P1    0      32     22      32        
P2    3      18     11      17        
P3    6      34     21      32        
P4    9      31     21      28        

Priority (Non-Preemptive):
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P4    16     23     13      20      

## **Gantt Charts and Averages**

In [16]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"]) #Sort tasks by start time for correct chart order

    chart = "" #For bar representation
    timeline = "" #For timestamp markers

    for r in results: #Build bars and timeline for each process
        length = r["finish"] - r["start"] #Duration of process execution
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}" #Add start time and spacing

    timeline += f"{results[-1]['finish']}" #Add final ending time

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results: #Create a line with PID labels centered in each bar
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-") #Center PID inside the block
    print(label_line + "|")

    print(timeline) #Print timeline with time markers

def compute_averages(results): #Compute average waiting & turnaround times
    total_wait = sum(r["waiting"] for r in results) #Sum of all waiting times
    total_turn = sum(r["turnaround"] for r in results) #Sum of all turnaround times
    n = len(results) #Number of processes
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n} # Average waiting and turnaround time

results = sjf(sample_processes)

print("SJF:")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = srtf(sample_processes)

print("\nSRTF:")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = fcfs(sample_processes)

print("FCFS Results:")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = round_robin(sample_processes, quantum=3)

print("\nRound Robin (q=3):")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = priority_non_preemptive(sample_processes)

print("\nPriority (Non-Preemptive):")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = priority_preemptive(sample_processes)

print("\nPriority (Preemptive):")
print_schedule(results)

make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

SJF:
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P4    16     23     13      20        
P3    23     34     21      32        

Gantt Chart:
|----------|------|-------|-----------|
|----P1----|--P2--|---P4--|-----P3----|
0          10      16       23           34

Average Waiting Time = 10.75
Average Turnaround Time = 19.25


SRTF:
PID   Start  Finish Waiting Turnaround
P1    0      23     13      23        
P2    1      7      0       6         
P4    7      14     4       11        
P3    23     34     21      32        

Gantt Chart:
|-----------------------|------|-------|-----------|
|-----------P1----------|--P2--|---P4--|-----P3----|
0                       1      7       23           34

Average Waiting Time = 9.5
Average Turnaround Time = 18.0

FCFS Results:
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P3    16     27     14      25 